# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('clickstream.csv', sep='\t')

In [3]:
# Making sure clickstream is indeed sorted by timestamp
(np.abs(df['timestamp'] - df['timestamp'].sort_values())).sum()

0

In [4]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, functions as F, Window
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-24 15:51:34,926 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [5]:
clicks = se.read.csv('hdfs:/data/clickstream.csv', sep='\t', header=True)
clicks.createOrReplaceTempView('clicks')
clicks.limit(5).toPandas()

,user_id,session_id,event_type,event_page,timestamp
0,562,507,page,main,1695584127
1,562,507,event,main,1695584134
2,562,507,event,main,1695584144
3,562,507,event,main,1695584147
4,562,507,wNaxLlerrorU,main,1695584154


In [6]:
def save_csv(data, path, limit=30, sep='\t',
             show=True, display=30, **kwargs):
    display = min(limit, display)

    data = [(x[0], x[1]) for x in data.take(limit)]

    df = pd.DataFrame(data, columns=['route', 'count'])
    df.to_csv(path, sep=sep, index=False, **kwargs)
    if show:
        return df.head(display)

def compare_csv(file1, file2, **kwargs):
    diff = pd.read_csv(file1, **kwargs).compare(pd.read_csv(file2, **kwargs), result_names=("left", "right"))
    if diff.empty:
        print('Dataframes match!')
        return True, None
    return False, diff

## SQL

In [7]:
sql_routes = se.sql('''
    WITH min_errors AS (
        SELECT user_id AS uid, session_id AS sid, MIN(INT(timestamp)) AS min_timestamp
        FROM clicks
        WHERE INSTR(event_type, 'error') > 0
        GROUP BY user_id, session_id
    ), no_errors AS (
        SELECT user_id, session_id, event_page, timestamp,
               lag(event_page, 1) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS prev_page
        FROM clicks
        LEFT JOIN min_errors ON user_id = uid AND session_id = sid
        WHERE min_timestamp IS NULL OR INT(timestamp) < INT(min_timestamp)
        ORDER BY timestamp
    ), clean AS (
        SELECT user_id, session_id, event_page, timestamp
        FROM no_errors
        WHERE prev_page IS NULL OR event_page != prev_page
        ORDER BY timestamp
    ), routes AS (
        SELECT user_id, session_id, MAX(route) AS route
        FROM (SELECT user_id, session_id,
                     COLLECT_LIST(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS route
              FROM clean) AS temp
        GROUP BY user_id, session_id
    )

    SELECT CONCAT_WS('-', route) as route, COUNT(route) as count
    FROM routes
    GROUP BY route
    ORDER BY count DESC, route DESC''') # LIMIT 30 is omitted intentionally

In [8]:
save_csv(sql_routes, 'sql.csv')

,route,count
0,main,8184
1,main-archive,1113
2,main-rabota,1047
3,main-internet,897
4,main-bonus,870
5,main-news,769
6,main-tariffs,677
7,main-online,587
8,main-vklad,518
9,main-rabota-archive,170


## RDD

In [9]:
rdd_min_errors = (clicks
                  .rdd
                  .filter(lambda x: 'error' in x.event_type)
                  .map(lambda x: ((x.user_id, x.session_id), int(x.timestamp)))
                  .reduceByKey(lambda v1, v2: min(v1, v2)))

rdd_no_errors = (clicks
                 .rdd
                 .map(lambda x: ((x.user_id, x.session_id), x))
                 .leftOuterJoin(rdd_min_errors)
                 .filter(lambda x: x[1][1] is None or int(x[1][0].timestamp) < x[1][1])
                 .map(lambda x: (x[0], x[1][0]))
                 .sortBy(lambda x: int(x[1].timestamp)))

def clean(vals):
    res = []
    for val in sorted(vals, key=lambda x: int(x.timestamp)):
        if len(res) == 0 or res[-1] != val.event_page:
            res.append(val.event_page)
    return '-'.join(res)

rdd_clean = (rdd_no_errors
             .groupByKey()
             .mapValues(lambda vals: clean(vals)))

rdd_routes = (rdd_clean
              .map(lambda x: (x[1], 1))
              .reduceByKey(lambda x, y: x + y)
              .sortBy(lambda x: (x[1], x[0]), ascending=False))

In [10]:
save_csv(rdd_routes, 'rdd.csv')

,route,count
0,main,8184
1,main-archive,1113
2,main-rabota,1047
3,main-internet,897
4,main-bonus,870
5,main-news,769
6,main-tariffs,677
7,main-online,587
8,main-vklad,518
9,main-rabota-archive,170


In [11]:
assert compare_csv('rdd.csv', 'sql.csv')[0]

Dataframes match!


## DF

In [12]:
# I intentionally didn't cast values before to make use of casts embedded into sql and rdd solutions
clicks = clicks.withColumn('user_id', F.col('user_id').cast('int'))
clicks = clicks.withColumn('session_id', F.col('session_id').cast('int'))
clicks = clicks.withColumn('timestamp', F.col('timestamp').cast('int'))
clicks

DataFrame[user_id: int, session_id: int, event_type: string, event_page: string, timestamp: int]

In [13]:
df_min_errors = (clicks
                 .where(F.col('event_type').contains('error'))
                 .groupBy(['user_id', 'session_id'])
                 .agg(F.min('timestamp').alias('min_timestamp'))
                 .select(['user_id', 'session_id', 'min_timestamp']))

df_no_errors = (clicks
                .join(df_min_errors, ['user_id', 'session_id'], 'left')
                .where(F.col('min_timestamp').isNull() | (F.col('timestamp') < F.col('min_timestamp')))
                .withColumn('prev_page', F.lag('event_page', 1).over(Window.partitionBy(['user_id', 'session_id']).orderBy('timestamp')))
                .orderBy('timestamp')
                .select(['user_id', 'session_id', 'event_page', 'timestamp', 'prev_page']))

df_clean = (df_no_errors
            .where(F.col('prev_page').isNull() | (F.col('event_page') != F.col('prev_page')))
            .orderBy('timestamp')
            .withColumn('route', F.collect_list('event_page').over(Window.partitionBy(['user_id', 'session_id']).orderBy('timestamp')))
            .select(['user_id', 'session_id', 'route']))

df_routes = (df_clean
             .groupBy(['user_id', 'session_id'])
             .agg(F.max('route').alias('route'))
             .groupBy('route')
             .agg(F.count('route').alias('count'))
             .orderBy(['count', 'route'], ascending=False)
             .withColumn('route', F.concat_ws('-', 'route')))

In [14]:
save_csv(df_routes, 'df.csv')

,route,count
0,main,8184
1,main-archive,1113
2,main-rabota,1047
3,main-internet,897
4,main-bonus,870
5,main-news,769
6,main-tariffs,677
7,main-online,587
8,main-vklad,518
9,main-rabota-archive,170


In [15]:
assert compare_csv('df.csv', 'sql.csv')[0]

Dataframes match!


In [16]:
# stop Spark (and YARN application)
sc.stop()